#<center>BERT Twitter 文書分類 </center>

###<center>参考サイト</center>

###<center>https://qiita.com/takubb/items/fd972f0ac3dba909c293</center>

1. 準備（ライブラリのインストール）
2. データセットの準備
3. データの前処理（BERTが受けとれる形式にデータを整形）
4. 学習済みモデルのロード
5. 訓練（Pre-trainedモデルのファインチューニング）
6. 検証

#0. 作業環境の準備

##0.1. google driveのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##0.2. カレントディレクトリへ移動

In [2]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


##0.3. GPUへ接続

In [3]:
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

##0.4. 現在時刻の取得

In [4]:
import datetime
import pytz

def pull_Datetime():
  DATETIME_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  #print(DATETIME_now)

  return DATETIME_now

##0.5. 分類モデルの性格指標を指定

この指標をもとに、2クラス分類を行う。

E or I なら
EI指標の分類器を作成する。

In [5]:
TYPE_INDEX='EorI_E'
#TYPE_INDEX='NorS_N'
#TYPE_INDEX='ForT_F'
#TYPE_INDEX='JorP_J'

##0.6. 学習するデータセットの指定

ツイッターの各投稿に、MBTIの16性格がラベリングされている。


In [6]:
#INPUT_CSV_v6='./Assets/Assets_Output/Tweet_Datasets/v6A_forBERT.csv'
INPUT_CSV_v6='./Assets/Assets_Output/Tweet_Datasets/v6A_forBERT_2021-10-01.csv'

#1. 準備 (ライブラリのインストール)

Hugging Face Library 'Transformer' のダウンロード

* transformers
* 最新のNLPモデルを利用できるライブラリ
* BERTの他にもGPT-2, RoBERTa, XLM, DistilBert, XLNet, T5, CTRL...などの最新モデルや100言語以上の学習済みモデル（Pre-trained model）を利用可能

In [7]:
!pip install transformers

!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

#参考サイト (https://qiita.com/takubb/items/fd972f0ac3dba909c293) では欠けているライブラリ
!pip install fugashi
!pip install ipadic

     |████████████████████████████████| 2.9 MB 3.0 MB/s 
     |████████████████████████████████| 3.3 MB 35.1 MB/s 
     |████████████████████████████████| 895 kB 39.8 MB/s 
     |████████████████████████████████| 636 kB 39.7 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-do

#2. データセットの準備

##2.1.準備したツイートデータをダウンロードする。

In [8]:
import pandas as pd
df=pd.read_csv('%s' % INPUT_CSV_v6, engine='python')

# データの確認
print(f'データサイズ： {df.shape}')

df.sample(10)

データサイズ： (10000, 12)


,Unnamed: 0,author_id,text,tweet_id,tweet_time,conversation_id,lang,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
18,19084,2.854499e+09,今日も一日早く本店に戻りたい,9.349266e+17,2017-11-26 23:27:29+00:00,9.349266e+17,ja,INTJ,I,N,T,J
9573,47105,3.013653e+09,移動しました。,8.746514e+17,2017-06-13 15:35:22+00:00,8.746514e+17,ja,INTP,I,N,T,P
2088,110021,5.839700e+07,運慶見てきた。一番最初に目に入って来た「運慶デビュー作」と題された像が一番良かった。初心を以...,9.139351e+17,2017-09-30 01:14:50+00:00,9.139351e+17,ja,INFJ,I,N,F,J
3622,403797,1.968939e+09,RT @y_motors: 【田村健太郎】\n年の瀬変愛ドラマ「暇な女子大生」に出演いたしま...,9.460747e+17,2017-12-27 17:46:06+00:00,9.460747e+17,ja,ISFP,I,S,F,P
9560,288507,8.036056e+17,RT @asagiri96mc: 熱盛！が話題なので、いつでもどこでもｱﾂﾓﾘｨｯ!!!!!...,9.014273e+17,2017-08-26 12:53:17+00:00,9.014273e+17,ja,ISTJ,I,S,T,J
5025,136363,7.577986e+17,礼装運は良かったんですけど回る気が起きなくてですね…ね… https://t.co/gD2O...,9.452532e+17,2017-12-25 11:21:46+00:00,9.452532e+17,ja,INFJ,I,N,F,J
9957,375720,2.911464e+09,はいディフューザー設置 https://t.co/KOvV9ufeIn,8.684785e+17,2017-05-27 14:46:36+00:00,8.684785e+17,ja,ISTP,I,S,T,P
6948,35364,2.258328e+09,RT @nez_9090: そんなアサシンみたいな目で見なくても…\n被り物より目つき鋭いん...,9.249485e+17,2017-10-30 10:37:55+00:00,9.249485e+17,ja,INTP,I,N,T,P
6606,69858,3.495948e+08,6月11日(日)までの期間限定！こざき亜衣先生の『あさひなぐ』22巻までアプリで今だけ読める...,8.719579e+17,2017-06-06 05:12:16+00:00,8.719579e+17,ja,INTP,I,N,T,P
6264,156170,2.396327e+09,RT @kinonotabianime: 今秋放送『キノの旅 -the Beautiful ...,8.905871e+17,2017-07-27 14:58:08+00:00,8.905871e+17,ja,INFP,I,N,F,P


##2.2. 性格指標のone-hot化

In [9]:
df=pd.get_dummies(df, columns=["label_EorI", "label_NorS", "label_ForT", "label_JorP"], sparse=True)
df

,Unnamed: 0,author_id,text,tweet_id,tweet_time,conversation_id,lang,MBTI_Type,label_EorI_E,label_EorI_I,label_NorS_N,label_NorS_S,label_ForT_F,label_ForT_T,label_JorP_J,label_JorP_P
0,332832,4.911959e+09,おかしいなぁ、うまいこと詠めたと思ったのは自分だけだったようだ…,9.415587e+17,2017-12-15 06:41:06+00:00,9.415587e+17,ja,ISFJ,0,1,0,1,1,0,1,0
1,136789,1.940916e+09,RT @katomayumi: 創作漫画「金髪ちゃんとマジメ君」ちょっと前に話題になった校則...,9.420128e+17,2017-12-16 12:45:31+00:00,9.420128e+17,ja,INFJ,0,1,1,0,1,0,1,0
2,23309,1.563000e+09,@hixna_cooking もうそんなに作ったのか\n壁|ω・`）開店まだかなー,8.546768e+17,2017-04-19 12:43:12+00:00,8.542972e+17,ja,INTP,0,1,1,0,0,1,0,1
3,339022,6.055648e+08,@kubotayokoyama こういうノリ好きだったんだな,9.463650e+17,2017-12-28 12:59:22+00:00,9.463644e+17,ja,ISFJ,0,1,0,1,1,0,1,0
4,179532,2.989807e+09,なんか「あんまりストーリーとか考察されると先の展開読まれそうで嫌だ」って人もいるらしいけど、...,9.455742e+17,2017-12-26 08:37:07+00:00,9.455742e+17,ja,INFP,0,1,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,312054,1.893659e+09,@t_1211t おう！\n\nこいよー！！,9.407661e+17,2017-12-13 02:11:31+00:00,9.402224e+17,ja,ISFJ,0,1,0,1,1,0,1,0
9996,265559,7.133224e+17,@p_a_555 フリーズして泣いてるなう,9.329828e+17,2017-11-21 14:43:37+00:00,9.329005e+17,ja,ENFJ,1,0,1,0,1,0,1,0
9997,390395,2.741809e+09,BrainWarsでフレンドランキングで8位になりました！ #BrainWars https...,9.456265e+17,2017-12-26 12:04:56+00:00,9.456265e+17,ja,ISTP,0,1,0,1,0,1,0,1
9998,247811,4.228360e+09,ハマーン様,9.466353e+17,2017-12-29 06:53:29+00:00,9.466353e+17,ja,INFP,0,1,1,0,1,0,0,1


In [10]:
sentences= df['text'].values #sentences = df.sentence.values
labels= df['label_%s' % TYPE_INDEX].values #labels = df.label.values

In [11]:
labels 

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...]
Fill: 0
IntIndex
Indices: array([   5,   11,   19, ..., 9980, 9993, 9996], dtype=int32)

In [12]:
print(len(sentences))

10000


#3. データの前処理（BERTが受けとれる形式にデータを整形）

BERTでは基本的に学習済みモデルを利用する為、そのモデルが読み込めるフォーマットにデータを変換する必要があります
具体的には、以下の4つ手続きが必要になります

1. BERT Tokenizerを用いて単語分割・IDへ変換
>* 学習済みモデルの作成時と同じtokenizer（形態素解析器）を利用する必要がある
>* 日本語ではMecabやJuman++を利用されることが多い

2. Special tokenの追加
>* 文章の最後に[SEP]という単語する
>* 文章のはじめに[CLS]という単語を追加する（分類問題に利用される）
3. 文章の長さの固定
>* BERTでは全ての文書の長さ（単語の数）を同じにする必要がある（1文章あたりの最大の単語数は512単語）
>* そこで、Padding/Truncatingを用いて固定長に変換する
>* Paddingとは、指定した長さに満たない文章を[Pad]という意味を持たない単語の埋める処理
>* Truncatingとは、指定した長さを超える単語を切り捨てること
4. Attention mask arrayの作成
>* [Padding]を0、それ以外のTokenを1とした配列
>* 一見ややこししそう見えますが、どれもTransformerのライブラリでサポートされているので、簡単に実行できます

また、以下のイメージも参考にモデルの全体像と照らし合わせながら前処理内容をみるとイメージが沸きやすいかもしれません

##3.1. BERT Tokenizerを用いて単語分割・IDへ変換

In [13]:
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

#念の為、確認
## テスト実行
# 元文章
print(' Original: ', sentences[0])
# Tokenizer
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
# Token-id
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

 Original:  おかしいなぁ、うまいこと詠めたと思ったのは自分だけだったようだ…
Tokenized:  ['おかし', '##い', 'な', '##ぁ', '、', 'うま', '##い', 'こと', '詠', '##め', 'た', 'と', '思っ', 'た', 'の', 'は', '自分', 'だけ', 'だっ', 'た', 'よう', 'だ', '...']
Token IDs:  [21634, 28457, 18, 31050, 6, 7080, 28457, 45, 10775, 28506, 10, 13, 4479, 10, 5, 9, 1040, 687, 308, 10, 124, 75, 3215]


##3.2. 次に文章の長さを固定する為に、文章あたりの最大単語数を確認します

ここでは、以下の方法で最大単語数を確認していますが、最大単語数は決め打ちで設定することも可能です

In [14]:
# print(sentences[15806])

In [15]:
from tqdm.notebook import tqdm
import numpy as np

# 最大単語数の確認
max_len = []
# 1文づつ処理
len_sentences = len(sentences)
for sent in tqdm(sentences):
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))
# 最大の値を確認
len_max=max(max_len)
#len_q0, len_q25, len_q50, len_q75, len_q100=np.percentile(max_len,  [0, 25, 50, 75, 100])
#len_q75=int(len_q75)
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

  0%|          | 0/10000 [00:00<?, ?it/s]

最大単語数:  132
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


##3.3. Tokenizerと最大単語数の確認がとれたので、全ての文章に一括して処理を行います

tokenizer.encode_plusを利用するとまとめて処理が行えます

In [16]:
from tqdm.notebook import tqdm

input_ids = []
attention_masks = []

# 1文づつ処理
for sent in tqdm(sentences):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = len_max,        # 文章の長さを固定（Padding/Trancatinating）#mayo
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

  0%|          | 0/10000 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


##labelをtensor型に変換

labelはpd.get_dummiesでone-hotにしてます

In [17]:
# tenosor型に変換
labels = torch.tensor(labels)

## BERTの前処理が正常に実行されたか確認

前処理は、

Special tokenの追加、文章の長さの固定、Attention mask arrayの作成

である。

In [18]:
# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  おかしいなぁ、うまいこと詠めたと思ったのは自分だけだったようだ…
Token IDs: tensor([    2, 21634, 28457,    18, 31050,     6,  7080, 28457,    45, 10775,
        28506,    10,    13,  4479,    10,     5,     9,  1040,   687,   308,
           10,   124,    75,  3215,     3,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
         

1つ目の文章の前処理結果です

元々、日本語テキストだった文章がID化されています

Token IDsの最初に単語IDはSpecial tokenの[CLS]を表し、後半の0埋めが[Pad]を示しています



```
Original:  旧式Macで禁断のパワーアップ！最新PCやソフトを一挙にチェック
Token IDs: tensor([    2, 18718,  8653,    12,  1763, 29135,     5, 20734,   679,  6215,
         3794,    49,  1604,    11, 24598,     7,  9398,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])
```



##3.5. データローダの適用とデータの分割

ここまでで、前処理が完了したので、90%を訓練データ、10％をテストデータに分割して、pytorchで学習を行う為に、データローダーへ変換しておきます

データローダーの説明は、pytorchの基本的な操作なので本記事では割愛します（簡単に言うと、データをバッチごとに分割して、学習を上手くやってくれるデータ型です）

In [19]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90%地点のIDを取得
# train_size = int(0.9 * len(dataset))
# val_size = len(dataset) - train_size
#train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# データセットを分割
from sklearn.model_selection import train_test_split
train_dataset, val_dataset  = train_test_split(dataset, train_size=0.8)
val_dataset, test_dataset  = train_test_split(val_dataset, train_size=0.8)

print('訓練データ数：{}'.format(len(train_dataset)))
print('検証データ数:　{} '.format(len(val_dataset )))
print('テストデータ数:　{} '.format(len(test_dataset)))

# データローダーの作成
batch_size = 64

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
test_dataloader = DataLoader(
            test_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

訓練データ数：8000
検証データ数:　1600 
テストデータ数:　400 


#4. 学習済みモデルのロード

huggingface transformerのBERTファインチューニングでは以下のタスクをサポートしています
全てBERT Pre-trainedモデルをベースとして学習し、出力層のみそれぞれのタスクに適した構成になっています

* BertModel
* BertForPreTraining
*   BertForMaskedLM
*   BertForNextSentencePrediction
*   BertForSequenceClassification
*   BertForTokenClassification
* BertForQuestionAnswering

公式ドキュメント.

## BertForSequenceClassification

今回は2クラス分類問題なのでBertForSequenceClassificationを利用します

BertForSequenceClassificationは**「BERT学習済みモデルの最後の層に分類用のレイヤー追加したネットワーク構成」**です

データを入力することで、学習済みモデル全体と未学習部分の分類機レイヤーの学習が行われます

デフォルトでは、モデル全体のパラメーターがファインチューニングされますが、学習させるレイヤーの設定変更も可能です

モデルロード時にnum_labelsを変更することで多クラス分類問題へも対応できます

In [20]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

# モデルをGPUへ転送
model.cuda()

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

学習済みモデルのロードが完了しました

次にこのモデルを手元のデータに合うようにファインチューニングを行います

#5. 訓練（Pre-trainedモデルのファインチューニング）

モデルのファインチューニングは通常のニューラルネットワークの学習と同様で、損失関数と最適化手法（Optimizer）を指定して、学習ループ（任意のバッチサイズとエポックを指定）を回します

BERTでは損失関数はすでに定義されているので、ハイパーパラメーターとして、最適化手法とバッチサイズをエポック数だけ指定します

ここでは、論文（BERT paper, Appendix A.3）に従って、最適化関数はAdamW（Adam Weight Decay fix）を利用し、ハイパーパラメーターは下記のように設定します

Batch size: 32
Learning rate: 2e-5
Epochs: 4

In [21]:
# 最適化手法の設定
optimizer = AdamW(model.parameters(), lr=2e-5)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = outputs.loss
        logits= outputs.logits
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():
                    outputs = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            loss = outputs.loss
            logits= outputs.logits
            val_loss += loss.item()
    return val_loss

学習に必要な関数が定義できたので、学習を行う

In [22]:
# 学習の実行
max_epoch = 10
train_loss_ = []
test_loss_ = []
import time
start = time.time()
epoch=0

# 学習途中の状態を読み込む。
# checkpoint = torch.load("./6_BERT_Huggingface/NS指標/model_NorS_N.tar")
# model.load_state_dict(checkpoint["model_state_dict"])
# optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
# epoch = checkpoint["epoch"]

for epoch in range(epoch, max_epoch):
    print("%s epoch" % (epoch))
    train_ = train(model)

    # 学習途中の状態を保存する。 #https://pystyle.info/pytorch-save-and-load-model/
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
         },
         "model_%s.tar" % TYPE_INDEX,
    )
  
    test_ = validation(model)
    train_loss_.append(train_)
    test_loss_.append(test_)
    
    elapsed_time = time.time() - start
    print("%s epoch  :  %s [sec] → %s" % (epoch, elapsed_time, pull_Datetime()))
    print()

print("終了時間 : %s [sec] → %s" % (elapsed_time, pull_Datetime()))

0 epoch
0 epoch  :  337.7420029640198 [sec] → 2021-10-01 23:34:03.160658+09:00

1 epoch
1 epoch  :  677.2112815380096 [sec] → 2021-10-01 23:39:42.589442+09:00

2 epoch
2 epoch  :  1016.0052318572998 [sec] → 2021-10-01 23:45:21.383380+09:00

3 epoch
3 epoch  :  1355.0349006652832 [sec] → 2021-10-01 23:51:00.413042+09:00

4 epoch
4 epoch  :  1694.5274357795715 [sec] → 2021-10-01 23:56:39.905581+09:00

5 epoch
5 epoch  :  2033.6322793960571 [sec] → 2021-10-02 00:02:19.010463+09:00

6 epoch
6 epoch  :  2372.631543159485 [sec] → 2021-10-02 00:07:58.009688+09:00

7 epoch
7 epoch  :  2711.9295167922974 [sec] → 2021-10-02 00:13:37.307669+09:00

8 epoch
8 epoch  :  3050.9318890571594 [sec] → 2021-10-02 00:19:16.310078+09:00

9 epoch
9 epoch  :  3389.929565668106 [sec] → 2021-10-02 00:24:55.307728+09:00

終了時間 : 3389.929565668106 [sec] → 2021-10-02 00:24:55.308675+09:00


In [23]:
train_loss_

[53.66926470398903,
 51.133606016635895,
 45.395859599113464,
 33.49273946136236,
 21.72710332274437,
 14.018326973542571,
 8.98321326635778,
 7.3449452868662775,
 5.012969349510968,
 4.970678620273247]

In [24]:
test_loss_

[9.972379744052887,
 9.980983197689056,
 10.816973328590393,
 13.044349730014801,
 14.27192708849907,
 18.06102028489113,
 21.940532565116882,
 21.1807658970356,
 24.446098387241364,
 26.46728801727295]

In [25]:
 test_

26.46728801727295

#6. 検証

In [26]:
# 検証方法の確認（1バッチ分で計算ロジックに確認）

model.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

以下の様な結果が出力されます
この確率の様な値（厳密には、この値ををソフトマックス関数に入力すると確率になる）が大きい方のラベルをモデルは分類結果とし予測としています

In [27]:
## 予測結果の確認
print(f'出力:{preds}')

出力:SequenceClassifierOutput(loss=None, logits=tensor([[ 4.3104, -3.9059],
        [ 0.8064, -0.9268],
        [ 3.8562, -3.7186],
        [-0.0276,  0.1159],
        [ 3.9209, -3.6104],
        [ 4.3326, -4.0139],
        [ 4.0426, -3.9256],
        [ 3.0647, -3.0585],
        [ 3.3782, -2.9615],
        [ 3.6342, -3.8006],
        [ 4.4756, -4.1187],
        [ 4.3835, -4.1691],
        [ 4.0125, -3.9002],
        [ 4.2488, -3.9564],
        [ 2.9832, -2.6729],
        [ 4.3202, -3.9361],
        [ 4.4464, -3.9970],
        [ 3.9867, -3.6583],
        [ 3.8736, -3.7766],
        [ 4.6104, -4.4727],
        [ 4.5327, -4.0508],
        [-2.5020,  2.3933],
        [ 1.6310, -1.5303],
        [ 3.8525, -3.4980],
        [-2.4389,  2.4431],
        [ 4.2408, -3.7102],
        [ 3.4451, -3.4183],
        [ 3.4376, -3.3883],
        [ 4.0393, -4.0032],
        [ 0.0163, -0.2807],
        [ 4.3784, -4.1660],
        [ 2.4830, -2.0864],
        [ 4.0622, -3.7939],
        [ 4.3971, -4.0863],
  



```
出力:(tensor([[-5.0226,  5.0193],
        [-5.0390,  4.9736],
        [ 4.7941, -4.7459],
        [-4.9395,  4.6827], device='cuda:0'),)
```



左からラベルの[0, 1]に対応しており、1つ目のデータであれば、右側の値が大きいのでモデルはラベル1を予測している

In [28]:
# 比較しやすい様にpd.dataframeへ整形
import numpy as np
import pandas as pd
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

accuracy_df.to_csv('./%s.csv' % TYPE_INDEX)

In [29]:
accuracy_df

,logit_0,logit_1,pred_label,true_label
0,4.310365,-3.905881,0,0
1,0.806414,-0.926764,0,0
2,3.856167,-3.718582,0,0
3,-0.027636,0.115929,1,0
4,3.920884,-3.610384,0,0
...,...,...,...,...
59,3.690884,-3.678268,0,0
60,4.512427,-4.079139,0,1
61,4.198460,-4.081662,0,0
62,2.010562,-2.183952,0,0


In [30]:
accuracy_df['answer'] = (accuracy_df['pred_label'] == accuracy_df['true_label'])
list_answer=accuracy_df['answer'].to_list()
count_true=list_answer.count(True)
print(list_answer)
print(count_true)
print('accuracy : ',  float(count_true / len(list_answer))*100)

[True, True, True, False, True, True, False, True, True, True, True, True, False, True, True, True, True, False, True, True, True, False, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, False, False, True, True, True, True, True, True, True, True, False, True, False, True, True, True]
53
accuracy :  82.8125


In [31]:
pred_df.values

array([[0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

上記の様に予測ラベルと正解ラベルを取得して、dataframeへ保存することができました

あとは、シンプルに正解率を計算するなり、sklearnを用いて混合行列を作成するなり、好きな様に分類モデルの評価を行うことができる。